# Tutorial 10: Traffic Lights

This tutorial walks through how to add traffic lights to experiments. This tutorial will use the following files:
本教程讲解如何将红绿灯添加到实验中。本教程将使用以下文件:

* Experiment config for RL version of traffic lights in gridRL版本交通信号灯在网格中的实验配置: `examples/exp_configs/rl/singleagent/singleagent_traffic_light_grid.py`
* Experiment config for non-RL version of traffic lights in grid非rl版本交通信号灯在网格中的实验配置: `examples/exp_configs/non_rl/traffic_light_grid.py`
* Network网络: `traffic_light_grid.py` (class TrafficLightGridScenario)
* Environment for RL version of traffic lights in grid环境为RL版本的交通信号灯在网格: (class TrafficLightGridEnv)
* Environment for non-RL version of traffic lights in grid非rl版本的网格交通信号灯环境: (class AccelEnv)

There are two main classes of traffic lights that Sumo supports: (1) actuated and (2) static traffic lights. This tutorial will cover both types. Moreover, in this tutorial, we'll discuss another type of traffic light. In total, we have 4 types of traffic lights in the Flow:
相扑主要支持两种类型的交通灯:(1)驱动型交通灯和(2)静态交通灯。本教程将涵盖这两种类型。此外，在本教程中，我们将讨论另一种类型的红绿灯。我们总共有4种交通信号灯:

1. Static Traffic Lights --> (Section 3)静态交通灯
2. Actuated Traffic Lights --> (Section 4)驱动交通灯
3. Actuated Baseline Traffic Lights --> (Section 5)启动基线交通灯
4. RL Traffic Lights --> (Section 6)RL交通灯

Let's begin!

First, import all necessary classes. 

In [ ]:
from flow.core.params import NetParams
from flow.networks.traffic_light_grid import TrafficLightGridNetwork
from flow.core.params import TrafficLightParams
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams, \
    InFlows, SumoCarFollowingParams
from flow.core.params import VehicleParams
import numpy as np

## 1. New parameters in `additional_net_params`

There are a few unique additions to `additional_net_params` in the traffic light grid environments to be aware of. They are the following 2 items:
“additional_net_params”在交通灯网格环境中有一些独特的附加功能需要注意。它们是以下两个项目:

#### grid_array
`grid_array` passes information on the road network to the network, specifying the parameters you see below: `row_num`, `col_num`, `inner_length`, `short_length`, `long_length`, `cars_top`, `cars_bot`, `cars_left`, `cars_right`. This is required for any traffic light grid experiment.

' grid_array '将道路网络上的信息传递给网络，指定如下参数:' row_num '、' col_num '、' inner_length '、' short_length '、' long_length '、' cars_top '、' cars_bot '、' cars_left '、' cars_right '。这是任何交通灯网格实验都需要的。



#### tl_logic
`tl_logic` should be used for users who want to exert more control over individual traffic lights. `tl_logic` simply tells the env whether the traffic lights are controlled by RL or whether a default pattern or SUMO actuation is to be used. Use "actuated" if you want SUMO to control the traffic lights. 

“tl_logic”应该用于希望对单个红绿灯施加更多控制的用户。“tl_logic”简单地告诉环境红绿灯是由RL控制的，还是使用默认模式或SUMO驱动。如果你想用相扑来控制交通灯，就用“启动”。

For this tutorial, we will assume the following parameters for the `grid_array`, which specifies a traffic light grid network with 2 rows and 3 columns. `traffic_lights` should be set to `True` for every experiment in this tutorial.

在本教程中，我们将假设' grid_array '的以下参数，它指定了一个具有2行3列的交通灯网格网络。本教程中的每个实验都应该将“traffic_lights”设置为“True”。

In [ ]:
inner_length = 300
long_length = 500
short_length = 300
n = 2 # rows
m = 3 # columns
num_cars_left = 20
num_cars_right = 20
num_cars_top = 20
num_cars_bot = 20
tot_cars = (num_cars_left + num_cars_right) * m \
    + (num_cars_top + num_cars_bot) * n

grid_array = {"short_length": short_length, "inner_length": inner_length,
              "long_length": long_length, "row_num": n, "col_num": m,
              "cars_left": num_cars_left, "cars_right": num_cars_right,
              "cars_top": num_cars_top, "cars_bot": num_cars_bot}

## 2. Defining Traffic Light Phases  界定交通灯的相位


To start off, we define how SUMO represents traffic light phases. A phase is defined as the states that the traffic lights around an intersection can take. The phase of a typical four-way, traffic-light-controlled intersection is modeled by a string (of length 4, 8, or 12, etc., depending on the structure of the intersection). 

首先，我们定义相扑如何表示红绿灯阶段。一个阶段被定义为一个十字路口周围的红绿灯所能达到的状态。一个典型的四路交通信号灯控制的交叉口的相位由一个字符串(长度为4、8或12等，取决于交叉口的结构)来建模。

Consider the phase "GrGr". Every letter in this phase string ("G", "r", "G", "r") corresponds to a signal of an edge in the intersection, in clockwise order (starting from the northbound). Explicitly, the northern and southern edges of the intersection both have a state of "G" (green), where the eastern and western edges of the intersection both have a state of "r" (red). In this example, the intersection has 4 edges, each edge has one lane, and the only possible direction is going straight. 

考虑阶段“GrGr”。这个相位串中的每一个字母(“G”、“r”、“G”、“r”)都对应着交叉点上一条边的信号，按顺时针方向排列(从北行开始)。明确地说，交叉口的南北侧的状态都是“G”(绿色)，而交叉口的东西侧的状态都是“r”(红色)。在这个例子中，十字路口有四条边，每条边都有一条车道，唯一可能的方向是直线。

Each character within a phase's state describes the state of one signal of the traffic light. Please note, that a single lane may contain several signals - for example one for vehicles turning left and one for vehicles which move straight (in this case, we may have something like "GgrrGgrr"). In other words, a signal does not control lanes, but links - each connecting a lane which is incoming into a junction to one which is outgoing from this junction.

相位状态中的每个字符描述一个交通灯信号的状态。请注意，单个车道可能包含多个信号——例如，一个用于左转的车辆，一个用于直行的车辆(在这种情况下，我们可能有类似“GgrrGgrr”的信号)。换句话说，信号并不是控制车道，而是连接——每个连接的车道从一个路口进入，到另一个路口离开。

For more information about traffic light states, please refer to 有关交通灯状态的更多信息，请参阅 https://sumo.dlr.de/wiki/Simulation/Traffic_Lights#Signal_state_definitions


NOTE: If the API is used at any point to modify the traffic light state, i.e. functions such as `setRedYellowGreenState`, this will override the traffic light's default phase.

注意:如果在任何时候使用API来修改交通灯状态，例如“setRedYellowGreenState”这样的函数，它将覆盖交通灯的默认阶段。

To do anything with traffic lights, you should interface with Flow's `TrafficLightParams` class

要处理交通灯，您应该与Flow的“TrafficLightParams”类交互

Once the `TrafficLightParams` class is instantiated, traffic lights can be added via the `add` function. One prerequisite of using this function is knowing the node id of any node you intend to manipulate. This information is baked into the experiment's network class, as well as the experiment's `nod.xml` file. For the experiment we are using with 2 rows and 3 columns, there are 6 nodes: "center0" to "center5". 

一旦实例化了“TrafficLightParams”类，就可以通过“add”函数添加交通灯。使用此函数的一个先决条件是了解要操作的任何节点的节点id。这些信息被纳入了实验的网络类，以及实验的“nod.xml"文件。对于我们使用的2行3列的实验，有6个节点:“center0”到“center5”。

This will be the ordering of "centers" in our network:

     | | |
    -3-4-5-
     | | |
    -0-1-2-
     | | |

In [ ]:
tl_logic = TrafficLightParams()

nodes = ["center0", "center1", "center2", "center3", "center4", "center5"]
phases = [{"duration": "31", "state": "GrGr"},
          {"duration": "6", "state": "yryr"},
          {"duration": "31", "state": "rGrG"},
          {"duration": "6", "state": "ryry"}]

In this particular example, each of the 6 intersections corresponds to the same set of possible phases; in other words, at any time, all intersections will be at the same phase in this example. 

在这个特殊的例子中，6个交叉点中的每一个都对应同一组可能的相位;换句话说，在任何时候，本例中的所有交叉点都处于相同的阶段。

In [ ]:
for node_id in nodes:
    tl_logic.add(node_id, tls_type="static", programID="1", offset=None, phases=phases)

You can, however, customize a network in which each traffic light node has different phases. 

但是，您可以自定义一个网络，其中每个红绿灯节点具有不同的阶段。

Following this step, the instance `tl_logic` of `TrafficLightParams` class should be passed into the network as element `traffic_lights`.

按照这个步骤，应该将“TrafficLightParams”类的实例“tl_logic”作为元素“traffic_lights”传递到网络中。

In [ ]:
additional_net_params = {"grid_array": grid_array, "speed_limit": 35,
                         "horizontal_lanes": 1, "vertical_lanes": 1,
                         "traffic_lights": True}
net_params = NetParams(additional_params=additional_net_params)

network = TrafficLightGridNetwork(name="grid",
                            vehicles=VehicleParams(),
                            net_params=net_params,
                            initial_config=InitialConfig(),
                            traffic_lights=tl_logic)

That's it! The traffic light logic will be passed into Flow's internals, which will generate an additional file containing all of the information needed to generate the traffic lights you specified in the simulation.

就是这样!交通灯逻辑将被传递到Flow的内部，它将生成一个额外的文件，其中包含生成仿真中指定的交通灯所需的所有信息。

## 3. Static Traffic Lights  静态交通灯

Static traffic lights are traffic lights with pre-defined phases. They cannot dynamically adjust according to the traffic needs; they simply follow the same pattern repeatedly. To see static traffic lights in action, the `TrafficLightParams` object should be instantiated with `baseline=False`. 

静态交通灯是具有预定相位的交通灯。不能根据流量需求动态调整;它们只是重复地遵循相同的模式。要查看静态交通灯的运行情况，应该使用' baseline=False '实例化' TrafficLightParams '对象。

When adding individual traffic lights, the following parameters in addition to `node_id` are involved:

在添加单个红绿灯时，除了‘node_id’外，还涉及以下参数:

* `tls_type`:  _[optional]_ str, specifies actuated or static traffic lights, defaults to static指定激活或静态交通灯，默认为静态
* `programID`:  _[optional]_ str, the program name for this traffic light. It cannot be the same ID as the base program, which is 0, defaults to 10这个红绿灯的程序名。它不能与基本程序相同的ID，后者为0，默认为10
* `offset`: _[optional]_ int, the initial time offset of the program int，程序的初始时间偏移量

An example of adding one static traffic light to our system is as follows在我们的系统中加入一个静态交通灯的例子如下:



In [ ]:
tl_logic = TrafficLightParams(baseline=False)
phases = [{"duration": "31", "state": "GrGr"},
          {"duration": "6", "state": "yryr"},
          {"duration": "31", "state": "rGrG"},
          {"duration": "6", "state": "ryry"}]
tl_logic.add("center0", phases=phases, programID=1)

## 4. Actuated Traffic Lights 驱动交通信号灯

For more flexibility than the static traffic lights defined above, and more control than RL-controlled traffic lights, actuated traffic lights are a good option to consider.
为了比上面定义的静态交通灯更灵活，也为了比rl控制的交通灯更容易控制，驱动型交通灯是一个不错的选择。

To explain the actuated traffic lights, we refer to an excerpt from SUMO's documentation: "SUMO supports gap-based actuated traffic control. This control scheme is common in Germany and works by prolonging traffic phases whenever a continuous stream of traffic is detected. It switches to the next phase after detecting a sufficent time gap between sucessive vehicles. This allows for better distribution of green-time among phases and also affects cycle duration in response to dynamic traffic conditions."

为了解释驱动型交通灯，我们参考了相扑文档中的一段摘录:“相扑支持基于间隙的驱动型交通控制。这种控制方案在德国很常见，只要检测到连续的交通流，就会延长交通阶段。在检测到吸力车辆之间足够的时间间隔后，切换到下一阶段。这样可以更好地分配不同阶段的绿灯时间，还可以根据动态交通条件影响周期持续时间。”

The difference between phases for static and actuated traffic lights is that actuated traffic light has two additional parameters in `phases`, namely `minDur` and `maxDur`, which describe the allowed range of time durations for each phase. `minDur` is the minimum duration the phase will be held for, and `masDur` is the maximum duration the phase will be held for.

静态交通灯和驱动交通灯的不同之处在于，驱动交通灯在“相位”中有两个额外的参数，即“minDur”和“maxDur”，它们描述了每个相位允许的持续时间范围。“minDur”是该阶段将保持的最小持续时间，而“masDur”是该阶段将保持的最大持续时间。

In addition to these parameters of `phases` and all the required parameters of static of traffic lights, the following optional parameters are involved. The default values are set by SUMO: 
除了这些“相位”参数和所有需要的交通信号灯静态参数外，还涉及到以下可选参数。默认值由SUMO设置:

* `maxGap`: _[optional]_ int, describes the maximum time gap between successive vehicle sthat will cause the current phase to be prolonged描述导致当前阶段延长的连续车辆之间的最大时间间隔
* `detectorGap`: _[optional]_ int, determines the time distance between the (automatically generated) detector and the stop line in seconds (at each lane's maximum speed)确定(自动生成的)检测器与停止线之间的时间距离(以秒为单位)(以每条车道的最大速度)
* `showDetectors`: _[optional]_ bool, toggles whether or not detectors are shown in sumo-guiv切换探测器是否显示在sumo-gui中
* `file`: _[optional]_ str, the file into which the detector shall write results检测器将结果写入其中的文件
* `freq`: _[optional]_ int, the period over which collected values shall be aggregated收集的价值应当汇总的期间

An example of adding two actuated traffic lights to our system is as follows. The first trafic lights corresponds to more custom control, while the second one specifies minimal control.
在我们的系统中添加两个激活的红绿灯的例子如下。第一个trafic灯对应更多的自定义控制，而第二个指定最小的控制。

In [ ]:
tl_logic = TrafficLightParams(baseline=False)
phases = [{"duration": "31", "minDur": "8", "maxDur": "45", "state": "GrGr"},
          {"duration": "6", "minDur": "3", "maxDur": "6", "state": "yryr"},
          {"duration": "31", "minDur": "8", "maxDur": "45", "state": "rGrG"},
          {"duration": "6", "minDur": "3", "maxDur": "6", "state": "ryry"}]

tl_logic.add("center1", 
             tls_type="actuated", 
             programID="1", 
             phases=phases, 
             maxGap=5.0, 
             detectorGap=0.9, 
             showDetectors=False)

tl_logic.add("center2",
             tls_type="actuated")


## 5. Actuated Baseline Traffic Lights 驱动基线交通灯

We have developed an actuated traffic light "baseline" that can be used for any experiments on a grid. This baseline uses actuated traffic lights (section 4), and has been fine-tuned on many iterations of experiments with varying parameters. The actual parameters are located in the `TrafficLightParams` class under the getter function `actuated_default()`. For reference, these values are:

我们已经开发了一种可以用于网格上的任何实验的驱动交通灯“基线”。这条基线使用了被激活的交通灯(第4节)，并且在多次不同参数的反复试验中进行了微调。实际的参数位于getter函数“actuated_default()”下的“TrafficLightParams”类中。这些数值为:

In [ ]:
tl_type = "actuated"
program_id = 1
max_gap = 3.0
detector_gap = 0.8
show_detectors = True
phases = [{"duration": "31", "minDur": "8", "maxDur": "45", "state": "GrGr"},
        {"duration": "6", "minDur": "3", "maxDur": "6", "state": "yryr"},
        {"duration": "31", "minDur": "8", "maxDur": "45", "state": "rGrG"},
        {"duration": "6", "minDur": "3", "maxDur": "6", "state": "ryry"}]

To see the actuated baseline traffic lights in action, simply initialize the TrafficLightParams class with the `baseline` argument set to `True`, and pass it into the `additional_net_params`. Nothing else needs to be done; no traffic lights need to be added. 

要查看实际运行的基线交通灯，只需将TrafficLightParams类的“基线”参数设置为“True”，并将其传递到“additional_net_params”中。其他什么都不需要做;不需要增加交通灯。


In [ ]:
tl_logic = TrafficLightParams(baseline=True)
additional_net_params = {"grid_array": grid_array, 
                         "speed_limit": 35,
                         "horizontal_lanes": 1, 
                         "vertical_lanes": 1,
                         "traffic_lights": True, 
                         "tl_logic": tl_logic}

## 6. Controlling Your Traffic Lights via RL 通过RL控制你的红绿灯

This is where we switch from the non-RL experiment script to the RL experiment. 
这就是我们从非RL实验脚本切换到RL实验脚本的地方。

To control traffic lights via RL, no `tl_logic` element is necessary. This is because the RL agent is controlling all the parameters you were able to customize in the prior sections. The `additional_net_params` should look something like this: 

要通过RL控制交通灯，不需要“tl_logic”元素。这是因为RL代理控制着您在前面几节中能够自定义的所有参数。“additional_net_params”应该是这样的:

In [ ]:
additional_net_params = {"speed_limit": 35, "grid_array": grid_array,
                         "horizontal_lanes": 1, "vertical_lanes": 1,
                         "traffic_lights": True}

This will enable the program to recognize all nodes as traffic lights. The experiment then gives control to the environment; we are using `TrafficLightGridEnv`, which is an environment created for applying RL-specified traffic light actions (e.g. change the state) via TraCI.

这将使程序能够识别所有节点作为红绿灯。然后，实验对环境进行控制;我们使用的是“TrafficLightGridEnv”，这是一个通过TraCI应用指定的rl红绿灯操作(例如改变状态)创建的环境。

This is all you need to run an RL experiment! It is worth taking a look at the `TrafficLightGridEnv` class to further understanding of the experiment internals. The rest of this tutorial is an optional walkthrough through the various components of `TrafficLightGridEnv`:

这就是运行RL实验所需要的全部内容!有必要看看“TrafficLightGridEnv”课程，以进一步了解实验的内部机制。本教程的其余部分是对“TrafficLightGridEnv”的各个组件的可选演练:

### Keeping Track of Traffic Light State 跟踪交通灯状态


Flow keeps track of the traffic light states (i.e. for each intersection, time elapsed since the last change, which direction traffic is flowing, and whether or not the traffic light is currently displaying yellow) in the following variables:

Flow在以下变量中跟踪交通灯状态(即每个交叉口，自上次改变后经过的时间，交通流向，以及交通灯当前是否显示为黄色):


In [ ]:
# keeps track of the last time the traffic lights in an intersection were allowed to change 
#记录十字路口交通灯最后一次允许改变的时间
# (the last time the lights were allowed to change from a red-green state to a red-yellow state).
#(最后一次允许信号灯从红-绿状态变为红-黄状态)
self.last_change = np.zeros((self.rows * self.cols, 1))
# keeps track of the direction of the intersection (the direction that is currently being allowed
#跟踪交点的方向(当前允许的方向)
# to flow. 0 indicates flow from top to bottom, and 1 indicates flow from left to right.)
#流。0表示从上到下的流动，1表示从左到右的流动。)
self.direction = np.zeros((self.rows * self.cols, 1))
# value of 1 indicates that the intersection is in a red-yellow state (traffic lights are red for 
#值1表示交叉口处于红黄状态(交通灯为红色表示)
# one way (e.g. north-south), while the traffic lights for the other way (e.g. west-east) are yellow.
#一条路(如南北方向)，而另一条路(如西、东方向)的交通灯是黄色的。
# 0 indicates that the intersection is in a red-green state.
#0表示交点处于红绿状态。
self.currently_yellow = np.zeros((self.rows * self.cols, 1))

* The variable `self.last_change` indicates the last time the lights were allowed to change from a red-green state to a red-yellow state.
*变量“self”。last_change '表示最后一次允许灯从红-绿状态更改为红-黄状态。
* The variable `self.direction` indicates the direction of the intersection, i.e. the direction that is currently being allowed to flow. 0 indicates flow from top to bottom, and 1 indicates flow from left to right.
*变量“self。direction”。指示交叉点的方向，即当前允许流动的方向。0表示从上到下的流程，1表示从左到右的流程。
* The variable `self.currently_yellow` with a value of 1 indicates that the traffic light is in a red-yellow state. 0 indicates that the traffic light is in a red-green state.
*变量“self”。currently_yellow '的值为1，表示交通灯处于红黄状态。0表示交通灯处于红绿状态。

`self.last_change` is contingent on an instance variable `self.min_switch_time`. This is a variable that can be set in `additional_env_params` with the key name `switch_time`. Setting `switch_time` enables more control over the RL experiment by preventing traffic lights from switching until `switch_time` timesteps have occurred. In practice, this can be used to prevent flickering.

的自我。last_change '取决于一个实例变量' self.min_switch_time '。这是一个可以在‘additional_env_params’中设置的变量，键名为‘switch_time’。通过设置' switch_time '，可以防止交通灯在' switch_time '之前切换，从而对RL实验有更多的控制。在实践中，这可以用来防止闪烁。


In [ ]:
additional_env_params = {"target_velocity": 50, "switch_time": 3.0}

### Elements of RL for Controlling Traffic Lights 控制交通灯的RL元件

#### Action Space 动作空间

The action space may be any set of actions the user wishes the agent to do. In this example, the action space for RL-controlled traffic lights directly matches the number of traffic intersections in the system. Each intersection (traffic light node) corresponds to an action. The action space is thus defined as:

操作空间可以是用户希望代理执行的任何一组操作。在本例中，rl控制的红绿灯的动作空间直接与系统中交通路口的数量相匹配。每个交叉口(红绿灯节点)对应一个动作。因此，动作空间定义为:

In [ ]:
@property
def action_space(self):
    if self.discrete:
            return Discrete(2 ** self.num_traffic_lights)
        else:
            return Box(
                low=0,
                high=1,
                shape=(self.num_traffic_lights,),
                dtype=np.float32)

In the case that the action space is discrete, we need 1-bit (that can be 0 or 1) for the action of each traffic light node. Hence, we need `self.num_traffic_lights` bits to represent the action space. To make a `self.num_traffic_lights`-bit number, we use the pyhton's `Discrete(range)`, and since we have `self.num_traffic_lights` bits, the `range` will be 2^`self.num_traffic_lights`.

在动作空间是离散的情况下，每个红绿灯节点的动作需要1位(可以是0或1)。因此，我们需要“自我”。表示动作空间的num_traffic_lights位。创造自我。num_traffic_lights ' -bit number，我们使用pyhton's ' Discrete(range)，因为我们有' self '。num_traffic_lights ' bits， ' range '将是2^ ' self.num_traffic_lights '。

In the case that the action space is continuous, we use a range (that is currently (0,1)) of numbers for each traffic light node. Hence, we will define `self.num_traffic_lights` "Boxes", each in the range (0,1). 
在动作空间是连续的情况下，我们为每个红绿灯节点使用一个数字范围(即当前的(0,1))。因此，我们将定义“自我”。num_traffic_lights的“盒子”，每个盒子的范围是(0,1)。

Note that the variable `num_traffic_lights` is actually the number of intersections in the grid system, not the number of traffic lights. Number of traffic lights in our example is 4 times the number of intersections

注意，变量“num_traffic_lights”实际上是网格系统中十字路口的数量，而不是交通灯的数量。在我们的例子中，交通灯的数量是十字路口数量的4倍

#### Observation Space 观测空间

The observation space may be any set of state information the user wishes to provide to the agent. This information may fully or partially describe the state of the environment. The existing observation space for this example is designed to be a fully observable state space with the following metrics. For all vehicle, we want to know its velocity, its distance (in [unit]) from the next intersection, and the unique edge it is traveling on. For each traffic light, we want to know its current state (i.e. what direction it is flowing), when it last changed, and whether it was yellow. 

观察空间可以是用户希望提供给代理的任何一组状态信息。这些信息可以完全或部分地描述环境的状态。本例中现有的观察空间被设计为具有以下度量的完全可观察状态空间。对于所有的车辆，我们想要知道它的速度，它到下一个交叉口的距离(单位)，以及它行驶的唯一的边。对于每一个红绿灯，我们都想知道它的当前状态(即它是朝哪个方向流动的)，它最后一次改变是什么时候，以及它是否是黄色的。

In [ ]:
@property
def observation_space(self):
    speed = Box(
            low=0,
            high=1,
            shape=(self.initial_vehicles.num_vehicles,),
            dtype=np.float32)
        dist_to_intersec = Box(
            low=0.,
            high=np.inf,
            shape=(self.initial_vehicles.num_vehicles,),
            dtype=np.float32)
        edge_num = Box(
            low=0.,
            high=1,
            shape=(self.initial_vehicles.num_vehicles,),
            dtype=np.float32)
        traffic_lights = Box(
            low=0.,
            high=1,
            shape=(3 * self.rows * self.cols,),
            dtype=np.float32)
        return Tuple((speed, dist_to_intersec, edge_num, traffic_lights))

Note that in the case that the observation space is not fully-observable (e.g. cannot observe all the vehicles in the system), the observation space should be changed to only include those state information that are observable (e.g. velocity of N closest vehicles to an intersection)
需要注意的是，如果观测空间不是完全可观测的(例如不能观测到系统中所有的车辆)，观测空间应改为只包含可观测的状态信息(例如，距离交叉口最近的N辆车的速度)。

#### State Space 状态空间

The state space collects the information that the `observation_space` specifies. There are helper functions that exist in the `TrafficLightGridEnv` to construct the state space. 

状态空间收集' observation_space '指定的信息。在“TrafficLightGridEnv”中存在用于构造状态空间的辅助函数。

In [ ]:
def get_state(self):
    # compute the normalizers
        grid_array = self.net_params.additional_params["grid_array"]
        max_dist = max(grid_array["short_length"],
                       grid_array["long_length"],
                       grid_array["inner_length"])

        # get the state arrays
        speeds = [
            self.k.vehicle.get_speed(veh_id) / self.k.network.max_speed()
            for veh_id in self.k.vehicle.get_ids()
        ]
        dist_to_intersec = [
            self.get_distance_to_intersection(veh_id) / max_dist
            for veh_id in self.k.vehicle.get_ids()
        ]
        edges = [
            self._convert_edge(self.k.vehicle.get_edge(veh_id)) /
            (self.k.network.network.num_edges - 1)
            for veh_id in self.k.vehicle.get_ids()
        ]

        state = [
            speeds, dist_to_intersec, edges,
            self.last_change.flatten().tolist(),
            self.direction.flatten().tolist(),
            self.currently_yellow.flatten().tolist()
        ]
        return np.array(state)

#### Reward 奖励

The agents in an RL network will learn to maximize a certain reward. This objective can be defined in terms of maximizing rewards or minimizing the penalty. In this example, we penalize the large delay and boolean actions that indicate a switch (with the negative sign).

RL网络中的代理将学习如何最大化某个奖励。这个目标可以定义为使奖励最大化或惩罚最小化。在本例中，我们将惩罚指示开关(带负号)的大延迟和布尔操作。

In [ ]:
def compute_reward(self, rl_actions, **kwargs):
        return - rewards.min_delay_unscaled(self) - rewards.boolean_action_penalty(rl_actions >= 0.5, gain=1.0)

#### Apply RL Actions

In the `_apply_rl_actions` function, we specify what actions our agents should take in the environment. In this example, the agents (traffic light nodes) decide based on the action value how to change the traffic lights.

在' _apply_rl_actions '函数中，我们指定了我们的代理应该在环境中执行哪些操作。在本例中，代理(交通灯节点)根据动作值决定如何更改交通灯。

In [1]:
def _apply_rl_actions(self, rl_actions):
        """See class definition."""
        # check if the action space is discrete
        if self.discrete:
            # convert single value to list of 0's and 1's
            rl_mask = [int(x) for x in list('{0:0b}'.format(rl_actions))]
            rl_mask = [0] * (self.num_traffic_lights - len(rl_mask)) + rl_mask
        else:
            # convert values less than 0.5 to zero and above 0.5 to 1. 0 
            # indicates that we should not switch the direction, and 1 indicates
            # that switch should happen
            rl_mask = rl_actions > 0.5

        # Loop through the traffic light nodes    
        for i, action in enumerate(rl_mask):
            if self.currently_yellow[i] == 1:  # currently yellow
                # Code to change from yellow to red
                ...
            else:
                # Code to change to yellow
                ...

These are the portions of the code that are hidden from the above code for shortening the code:
这些部分的代码是隐藏在上述代码缩短代码:

In [ ]:
                # Code to change from yellow to red
                self.last_change[i] += self.sim_step
                # Check if our timer has exceeded the yellow phase, meaning it
                # should switch to red
                if self.last_change[i] >= self.min_switch_time:
                    if self.direction[i] == 0:
                        self.k.traffic_light.set_state(
                            node_id='center{}'.format(i),
                            state="GrGr")
                    else:
                        self.k.traffic_light.set_state(
                            node_id='center{}'.format(i),
                            state='rGrG')
                    self.currently_yellow[i] = 0

In [ ]:
                    # Code to change to yellow
                    if action:
                    if self.direction[i] == 0:
                        self.k.traffic_light.set_state(
                            node_id='center{}'.format(i),
                            state='yryr')
                    else:
                        self.k.traffic_light.set_state(
                            node_id='center{}'.format(i),
                            state='ryry')
                    self.last_change[i] = 0.0
                    self.direction[i] = not self.direction[i]
                    self.currently_yellow[i] = 1